# Equations of motion for segbot

This notebook makes use of [SymPy](https://docs.sympy.org/), which is a Python library for symbolic mathematics.

In [2]:
import sympy as sym
import numpy as np
from scipy import linalg
from scipy import signal

Define physical constants, consistent with the URDF file:

In [2]:
# Dimensions of chassis
dx = 0.4
dy = 0.6
dz = 0.8

# Distance between axle and COM of chassis
h = 0.3

# Half-distance between wheels
a = 0.7 / 2

# Mass of chassis
mb = 12.

# MOI of chassis
Jbx = (mb / 12) * (dy**2 + dz**2)
Jby = (mb / 12) * (dx**2 + dz**2)
Jbz = (mb / 12) * (dx**2 + dy**2)

# Radius of each wheel
r = 0.325

# Width of each wheel
hw = 0.075

# Mass of each wheel
mw = 1.2

# MOI of each wheel
Jw = (mw / 2) * r**2
Jwt = (mw / 12) * (3 * r**2 + hw**2)

# Total mass
m = mb + 2 * mw

# Total MOI
Jx = Jbx + 2 * Jwt
Jy = Jby
Jz = Jbz + 2 * Jwt

# Station parameters
station_velocity = -0.5 # <-- FIXME (change the velocity to change gravity)
station_radius = 20.    # <-- radius in meters of inside surface of station

# Acceleration of artifical gravity
g = station_velocity**2 * station_radius

Convert all physical constants to rational numbers:

In [1]:
# Dimensions
h = sym.nsimplify(h)
a = sym.nsimplify(a)
r = sym.nsimplify(r)

# Masses
mb = sym.nsimplify(mb)
mw = sym.nsimplify(mw)
m = sym.nsimplify(m)

# MOIs
Jx = sym.nsimplify(Jx)
Jy = sym.nsimplify(Jy)
Jz = sym.nsimplify(Jz)
Jw = sym.nsimplify(Jw)
Jwt = sym.nsimplify(Jwt)

# Gravity
g = sym.nsimplify(g)

NameError: name 'sym' is not defined

Define variables (with a flag to indicate they are all real numbers — this helps simplify):

In [4]:
(e_l,
 e_h,
 v,
 w,
 theta,
 thetadot,
 tau_R,
 tau_L) = sym.symbols('e_l, e_h, v, w, theta, thetadot, tau_R, tau_L', real=True)

In [5]:
thetadot

thetadot

Compute equations of motion, excluding lateral and heading errors (see [Tuttle, 2014](https://docs.lib.purdue.edu/cgi/viewcontent.cgi?article=1415&context=open_access_theses)):

In [6]:
M = sym.Matrix([[m + 2 * Jw / r**2, 0, mb * h * sym.cos(theta)],
                [0, (Jx + mb * h**2) * sym.sin(theta)**2 + Jz * sym.cos(theta)**2 + (2 * Jw * a**2 / r**2) + 2 * mw * a**2, 0],
                [mb * h * sym.cos(theta), 0, Jy * mb * h**2]])
N = sym.Matrix([[mb * h * (w**2 + thetadot**2) * sym.sin(theta)],
                [-2 * (Jx - Jz + m * h**2) * sym.cos(theta) * sym.sin(theta) * w * thetadot - mb * h * sym.sin(theta) * v * w],
                [(Jx - Jz + mb * h**2) * sym.cos(theta) * sym.sin(theta) * w**2 + mb * g * h * sym.sin(theta)]])
R = sym.Matrix([[1 / r, 1 / r],
                [-a / r, a / r],
                [-1, -1]])
f = sym.simplify(M.inv() * (N + R * sym.Matrix([[tau_L], [tau_R]])))

In [7]:
M

Matrix([
[           78/5,                                                            0, 18*cos(theta)/5],
[              0, 4289*sin(theta)**2/2000 + 1169*cos(theta)**2/2000 + 441/1000,               0],
[18*cos(theta)/5,                                                            0,         108/125]])

In [8]:
N

Matrix([
[                      (18*thetadot**2/5 + 18*w**2/5)*sin(theta)],
[-444*thetadot*w*sin(theta)*cos(theta)/125 - 18*v*w*sin(theta)/5],
[               39*w**2*sin(theta)*cos(theta)/25 + 18*sin(theta)]])

In [9]:
R

Matrix([
[ 40/13, 40/13],
[-14/13, 14/13],
[    -1,    -1]])

In [10]:
f

Matrix([
[        -(1200*tau_L + 1200*tau_R + 1404*(thetadot**2 + w**2)*sin(theta) + 65*(50*tau_L + 50*tau_R - 39*w**2*sin(2*theta) - 900*sin(theta))*cos(theta)/2)/(5850*cos(theta)**2 - 6084)],
[                                                                    32*(-875*tau_L + 875*tau_R - 1443*thetadot*w*sin(2*theta) - 2925*v*w*sin(theta))/(13*(3120*sin(theta)**2 + 2051))],
[5*(4225*tau_L + 4225*tau_R - 6591*w**2*sin(2*theta)/2 + 30*(100*tau_L + 100*tau_R + 117*(thetadot**2 + w**2)*sin(theta))*cos(theta) - 76050*sin(theta))/(702*(25*cos(theta)**2 - 26))]])

Compute full equations of motion:

In [11]:
f = sym.Matrix([[v * sym.sin(e_h)],
                [w],
                [f]])

Display the vector-valued function $f$ for which the equations of motion can be written as

$$\begin{bmatrix} \dot{e}_\text{lateral} \\ \dot{e}_\text{heading} \\ \dot{v} \\ \dot{w} \\ \ddot{\theta} \end{bmatrix} = f(e_\text{lateral}, e_\text{heading}, v, w, \theta, \dot{\theta}, \tau_R, \tau_L)$$

In [12]:
f

Matrix([
[                                                                                                                                                                           v*sin(e_h)],
[                                                                                                                                                                                    w],
[        -(1200*tau_L + 1200*tau_R + 1404*(thetadot**2 + w**2)*sin(theta) + 65*(50*tau_L + 50*tau_R - 39*w**2*sin(2*theta) - 900*sin(theta))*cos(theta)/2)/(5850*cos(theta)**2 - 6084)],
[                                                                    32*(-875*tau_L + 875*tau_R - 1443*thetadot*w*sin(2*theta) - 2925*v*w*sin(theta))/(13*(3120*sin(theta)**2 + 2051))],
[5*(4225*tau_L + 4225*tau_R - 6591*w**2*sin(2*theta)/2 + 30*(100*tau_L + 100*tau_R + 117*(thetadot**2 + w**2)*sin(theta))*cos(theta) - 76050*sin(theta))/(702*(25*cos(theta)**2 - 26))]])

Note that one of these ODEs is second-order — you will have to replace this with a set of two first-order ODEs, as usual.

In [13]:
f[0]

v*sin(e_h)

In [14]:
f[1]

w

In [15]:
f[2]

-(1200*tau_L + 1200*tau_R + 1404*(thetadot**2 + w**2)*sin(theta) + 65*(50*tau_L + 50*tau_R - 39*w**2*sin(2*theta) - 900*sin(theta))*cos(theta)/2)/(5850*cos(theta)**2 - 6084)

In [16]:
f[3]

32*(-875*tau_L + 875*tau_R - 1443*thetadot*w*sin(2*theta) - 2925*v*w*sin(theta))/(13*(3120*sin(theta)**2 + 2051))

In [17]:
f[4]

5*(4225*tau_L + 4225*tau_R - 6591*w**2*sin(2*theta)/2 + 30*(100*tau_L + 100*tau_R + 117*(thetadot**2 + w**2)*sin(theta))*cos(theta) - 76050*sin(theta))/(702*(25*cos(theta)**2 - 26))

In [18]:
phi, phidot = sym.symbols('phi, phidot')

In [19]:
phidot

phidot

In [20]:
fnew = sym.Matrix([[f[0]], [f[1]], [f[2]], [f[3]], [phi], [f[4]]])

fnew, fnew.shape

(Matrix([
 [                                                                                                                                                                           v*sin(e_h)],
 [                                                                                                                                                                                    w],
 [        -(1200*tau_L + 1200*tau_R + 1404*(thetadot**2 + w**2)*sin(theta) + 65*(50*tau_L + 50*tau_R - 39*w**2*sin(2*theta) - 900*sin(theta))*cos(theta)/2)/(5850*cos(theta)**2 - 6084)],
 [                                                                    32*(-875*tau_L + 875*tau_R - 1443*thetadot*w*sin(2*theta) - 2925*v*w*sin(theta))/(13*(3120*sin(theta)**2 + 2051))],
 [                                                                                                                                                                                  phi],
 [5*(4225*tau_L + 4225*tau_R - 6591*w**2*sin(2*theta)/2 + 30

In [21]:
fnew

Matrix([
[                                                                                                                                                                           v*sin(e_h)],
[                                                                                                                                                                                    w],
[        -(1200*tau_L + 1200*tau_R + 1404*(thetadot**2 + w**2)*sin(theta) + 65*(50*tau_L + 50*tau_R - 39*w**2*sin(2*theta) - 900*sin(theta))*cos(theta)/2)/(5850*cos(theta)**2 - 6084)],
[                                                                    32*(-875*tau_L + 875*tau_R - 1443*thetadot*w*sin(2*theta) - 2925*v*w*sin(theta))/(13*(3120*sin(theta)**2 + 2051))],
[                                                                                                                                                                                  phi],
[5*(4225*tau_L + 4225*tau_R - 6591*w**2*sin(2*theta)/2 + 30*(100*t

## Equilibrium Point Finding

In [22]:
taul, taur = sym.symbols('taul, taur')

In [23]:
dotv = (-2400*taul + 2400*taur + 65*(50*taul + 50*taur)) / (11700-12168)
dotw = (32*(-875*taul + 875*taur)) / (13*2051)
dotphi = (5*(8450*taul + 8450*taur+60*(100*taul + 100*taur))) / (1404*(25-26))

dotv_eq = sym.Eq(dotv, 0)
dotw_eq = sym.Eq(dotw, 0)
dotphi_eq = sym.Eq(dotphi, 0)

solved = sym.solve((vdot, wdot, dottheta), (taul, taur))
solved

NameError: name 'vdot' is not defined

In [24]:
e_le = 0
e_he = 0 # angle
ve = 2
we = 0
phie = 0 # angle
phidote = 0
tau_Re = 0
tau_Le = 0

## A, B Values

In [25]:
A_num = sym.lambdify((e_l,e_h, v, w, theta, thetadot, tau_L, tau_R), fnew.jacobian([e_l, e_h, v, w, theta, thetadot]))
B_num = sym.lambdify((e_l,e_h, v, w, theta, thetadot, tau_L, tau_R), fnew.jacobian([tau_L, tau_R]))

A = A_num(e_le, e_he, ve, we, phie, phidote, tau_Re, tau_Le)
B = B_num(e_le, e_he, ve, we, phie, phidote, tau_Re, tau_Le)

A = A.astype(float)
B = B.astype(float)

A, B

(array([[   0.        ,    2.        ,    0.        ,    0.        ,
            0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    1.        ,
            0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ,
         -125.        ,   -0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ,
            0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ,
            0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,   -0.        ,
          541.66666667,   -0.        ]]),
 array([[  0.        ,   0.        ],
        [  0.        ,   0.        ],
        [ 12.07264957,  12.07264957],
        [ -1.05014439,   1.05014439],
        [  0.        ,   0.        ],
        [-51.46011396, -51.46011396]]))

In [26]:
A.shape, B.shape

((6, 6), (6, 2))

## Finding K Value (A: 5x6, B:5x2), deprecated

In [27]:
## F = A - B@K, K: 2x6 matrix

import random

check = True
k = np.array([[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
attempts = 0
while check:
    if (attempts == 500000):
        print("500,000 attempts")
    elif (attempts == 1000000):
        print("1,000,000 attempts")
    elif (attempts == 2000000):
        print("2,000,000 attempts")
    elif (attempts == 3000000):
        print("3,000,000 attempts")
    elif (attempts == 4000000):
        print("4,000,000 attempts")
    elif (attempts == 5000000):
        print("5,000,000 attempts")
    elif (attempts == 6000000):
        print("6,000,000 attempts")
    elif (attempts == 7000000):
        print("7,000,000 attempts")
    elif (attempts == 8000000):
        print("8,000,000 attempts")
    elif (attempts == 9000000):
        print("9,000,000 attempts")
    elif (attempts == 20000000):
        print("20,000,000 attempts")
    elif (attempts == 50000000):
        print("50,000,000 attempts")
    elif (attempts == 100000000):
        print("100,000,000 attempts")
        
    a = random.uniform(-5,5)
    b = random.uniform(-5,5)
    c = random.uniform(-5,5)
    d = random.uniform(-5,5)
    e = random.uniform(-5,5)
    f = random.uniform(-5,5)
    g = random.uniform(-5,5)
    h = random.uniform(-5,5)
    i = random.uniform(-5,5)
    j = random.uniform(-5,5)
    l = random.uniform(-5,5)
    m = random.uniform(-5,5)
    k = np.array([[a, b, c, d, e, f], [g, h, i, j, l, m]])
    vals = np.linalg.eigvals(A-B@k)
    if (vals[0] < 0 and vals[1] < 0 and vals[2] < 0 and vals[3] < 0 and vals[4] < 0 and vals[5] < 0):
        check = False
    attempts += 1
print(k)
print(np.linalg.eigvals(A-B@k))


KeyboardInterrupt: 

## Finding K the Right Way

In [28]:
A[4][5] = 1

In [29]:
Am = sym.Matrix(A)
Bm = sym.Matrix(B)

Am, Bm

(Matrix([
 [0, 2.0, 0,   0,                0,   0],
 [0,   0, 0, 1.0,                0,   0],
 [0,   0, 0,   0,           -125.0,   0],
 [0,   0, 0,   0,                0,   0],
 [0,   0, 0,   0,                0, 1.0],
 [0,   0, 0,   0, 541.666666666667,   0]]),
 Matrix([
 [                0,                0],
 [                0,                0],
 [ 12.0726495726496, 12.0726495726496],
 [-1.05014439485429, 1.05014439485429],
 [                0,                0],
 [ -51.460113960114, -51.460113960114]]))

In [30]:
p = [-1, -3, -5, -7, -9, -11]

K = signal.place_poles(A, B, p).gain_matrix
K

array([[-52.48457626, -60.38127491,   8.77676097,  -9.83007396,
          1.58509775,   2.54172409],
       [ 52.90443651,  60.70596679,  -9.28829845,   9.85713386,
        -13.33220012,  -2.95954675]])

In [31]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @ B.T @ P
    return K, P

## Q Matrix, R Matrix Depiction

In [32]:
q1, q2, q3, q4, q5, q6 = sym.symbols('Q_1, Q_2, Q_3, Q_4, Q_5, Q_6')
Qs = np.diag([q1, q2, q3, q4, q5, q6])
Qs = sym.Matrix(Qs)
Qs

Matrix([
[Q_1,   0,   0,   0,   0,   0],
[  0, Q_2,   0,   0,   0,   0],
[  0,   0, Q_3,   0,   0,   0],
[  0,   0,   0, Q_4,   0,   0],
[  0,   0,   0,   0, Q_5,   0],
[  0,   0,   0,   0,   0, Q_6]])

In [33]:
5e3

5000.0

In [34]:
Q = np.diag([5e3, 10e3, 0.1, 50, 3000, 200])
R = np.eye(2)

In [35]:
Q, R

(array([[5.e+03, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00],
        [0.e+00, 1.e+04, 0.e+00, 0.e+00, 0.e+00, 0.e+00],
        [0.e+00, 0.e+00, 1.e-01, 0.e+00, 0.e+00, 0.e+00],
        [0.e+00, 0.e+00, 0.e+00, 5.e+01, 0.e+00, 0.e+00],
        [0.e+00, 0.e+00, 0.e+00, 0.e+00, 3.e+03, 0.e+00],
        [0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 2.e+02]]),
 array([[1., 0.],
        [0., 1.]]))

In [36]:
lqr(A, B, Q, R)

(array([[-50.        , -83.95766381,  -0.2236068 , -10.24444656,
         -44.46849183, -10.09570944],
        [ 50.        ,  83.95766381,  -0.2236068 ,  10.24444656,
         -44.46849183, -10.09570944]]),
 array([[ 4.19788319e+03,  1.02444466e+03, -3.55952008e-13,
          4.76125000e+01,  3.53145317e-14, -8.63914208e-14],
        [ 1.02444466e+03,  1.62497460e+03, -2.02761531e-14,
          7.99486854e+01,  1.05424476e-13, -5.41691422e-15],
        [-3.55952008e-13, -2.02761531e-14,  8.44517717e+00,
         -7.22144127e-15,  4.51493852e+00,  1.98560134e+00],
        [ 4.76125000e+01,  7.99486854e+01, -7.22144127e-15,
          9.75527424e+00,  5.24854862e-14, -1.78937145e-15],
        [ 3.53145317e-14,  1.05424476e-13,  4.51493852e+00,
          5.24854862e-14,  7.87492641e+02,  1.92334907e+00],
        [-8.63914208e-14, -5.41691422e-15,  1.98560134e+00,
         -1.78937145e-15,  1.92334907e+00,  6.62011333e-01]]))

## Randomizing Again

In [37]:
## F = A - B@K, K: 2x6 matrix

import random

check = True
k = np.array([[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
attempts = 0
while check:        
    a = random.uniform(-5,5)
    b = random.uniform(-5,5)
    c = random.uniform(-5,5)
    d = random.uniform(-5,5)
    e = random.uniform(-5,5)
    f = random.uniform(-5,5)
    g = random.uniform(-5,5)
    h = random.uniform(-5,5)
    i = random.uniform(-5,5)
    j = random.uniform(-5,5)
    l = random.uniform(-5,5)
    m = random.uniform(-5,5)
    k = np.array([[a, b, c, d, e, f], [g, h, i, j, l, m]])
    vals = np.linalg.eigvals(A-B@k)
    if (vals[0] < 0 and vals[1] < 0 and vals[2] < 0 and vals[3] < 0 and vals[4] < 0 and vals[5] < 0):
        check = False
    attempts += 1
print(np.linalg.eigvals(A-B@k))
k

[-6.20942151e+01+0.j         -2.03594548e+01+0.j
 -6.57016914e-01+1.26090682j -6.57016914e-01-1.26090682j
 -1.83807053e-02+0.04299002j -1.83807053e-02-0.04299002j]


array([[-1.84668245, -0.04658934, -0.19915093, -3.14743204,  0.38759545,
         1.75597872],
       [-0.39857961, -1.9604998 , -0.04972487, -2.79497859, -4.50691723,
        -3.43570544]])